## Imports

https://github.com/GeneralMills/pytrends


In [1]:
import pandas as pd
import numpy as np
import time
import requests
import json
from bs4 import BeautifulSoup
import random
import time
from tqdm import tqdm #https://pypi.org/project/tqdm/#ipython-jupyter-integration
#from tqdm import tqdm_notebook as tqdm #[If run in Jupyter Notebook]
import matplotlib.pyplot as plt
#from matplotlib.lines import Line2D½
%matplotlib inline
import seaborn as sns
import re	
from datetime import datetime as dt

from pytrends.request import TrendReq #pip install pytrends  | pip install pytrends --upgrade

from func import (chunks, reindex, global_id, term_list, time_corr_plot, trends_fetch, GT_dict, trends_fetch_cross)

# Pytrends

https://trends.google.com/trends/explore?date=all&geo=DK&q=kontanthj%C3%A6lp,Dagpenge,A-kasse,ASE,3F

For specifying conditions on terms: https://support.google.com/trends/answer/4359582?hl=en&ref_topic=4365599

Sources for SE terms:
- https://www.norden.org/da/info-norden/kontanthjaelp-i-sverige
- https://www.sverigesakassor.se/om-so/in-english/
- https://www.thenewbieguide.se/work/labour-legislation/labour-unions/

Sources for NO terms:
- https://www.norden.org/da/info-norden/norske-dagpenge-ved-ledighed
- https://www.norden.org/en/info-norden/trade-unions-norway
- https://www.jobboardfinder.com/search/best-job-site-in-norway

## Geo codes

Test queries to get geo codes

#### Denmark

In [ ]:
# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq(hl='DK', tz=360, timeout=(2, 5), retries=3, backoff_factor=1)

In [ ]:
# Create payload and capture API tokens. Only needed for interest_over_time(), interest_by_region() & related_queries()
pytrend.build_payload(kw_list=['dagpenge + "dagpenge regler" + dagpengeregler'], timeframe='2007-01-01 2007-02-01', geo='DK')

In [ ]:
# Interest by Region - percentage shares
interest_by_region_df = pytrend.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
interest_by_region_df

In [ ]:
#Manually labelling the regions
geo_regions_DK = {'DK-81':'North Denmark',
                  'DK-82':'Central Denmark',
                  'DK-83':'Southern Denmark',
                  'DK-84':'Capital',
                  'DK-85':'Zealand'}

#### Sweden

In [ ]:
# Create payload and capture API tokens. Only needed for interest_over_time(), interest_by_region() & related_queries()
pytrend.build_payload(kw_list=['jobcenter'], timeframe='all', geo='SE')

In [ ]:
# Interest by Region - percentage shares
interest_by_region_df = pytrend.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
#Dict in the same format as DK - no manual labelling
    #Removing County
geo_regions_SE = {y:x.replace(' County', '') for x,y in interest_by_region_df['geoCode'].to_dict().items()}

In [ ]:
geo_regions_SE

#### Norway

In [ ]:
# Create payload and capture API tokens. Only needed for interest_over_time(), interest_by_region() & related_queries()
pytrend.build_payload(kw_list=['jobcenter'], timeframe='all', geo='NO')

In [ ]:
# Interest by Region - percentage shares
interest_by_region_df = pytrend.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=True)

In [ ]:
#Dict in the same format as DK - no manual labelling
geo_regions_NO = {y:x for x,y in interest_by_region_df['geoCode'].to_dict().items()}

In [ ]:
geo_regions_NO

## Scrape

### Full query - DK

In [ ]:
dfTrends_DK = trends_fetch(geo_codes=geo_regions_DK, terms=term_list(geo='DK'), sleep=15)

In [ ]:
dfTrends_DK.shape

In [ ]:
dfTrends_DK.isna().sum()

In [ ]:
dfTrends_DK.to_csv('data/gt/dfTrends_DK_raw.csv', index=True, sep=';', encoding='utf-8-sig')

### Full query - SE

In [ ]:
term_list(geo = 'SE')

In [ ]:
dfTrends_SE = trends_fetch(geo_codes=geo_regions_SE, terms=term_list(geo='SE'), sleep=10)

In [ ]:
dfTrends_SE.shape

In [ ]:
# Removing addtional index row ?!
#dfTrends_SE.drop(['index'], axis = 1, inplace = True)

In [ ]:
dfTrends_SE.isna().sum()

In [ ]:
dfTrends_SE.to_csv('data/gt/dfTrends_SE_raw.csv', index=True, sep=';', encoding='utf-8-sig')

### Full query - NO

In [ ]:
term_list(geo = 'NO')

In [ ]:
dfTrends_NO =  trends_fetch(geo_codes=geo_regions_NO, terms=term_list(geo='NO'), sleep = 10)

In [ ]:
dfTrends_NO.shape

In [ ]:
# Removing addtional index row ?!
# dfTrends_NO.drop(['index'], axis = 1, inplace = True)

In [ ]:
dfTrends_NO.isna().sum()

In [ ]:
dfTrends_NO.to_csv('data/gt/dfTrends_NO_raw.csv', index=True, sep=';', encoding='utf-8-sig')

## Cross-sectional scrape attempt

#### Date lists

In [2]:
date_range_1 = pd.date_range(start = '2006-01-01', end = '2019-12-01', freq ='1M')-pd.offsets.MonthBegin(1)
date_range_2 = pd.date_range(start = '2006-02-01', end = '2019-12-01', freq ='1M')-pd.offsets.MonthBegin(1)

In [3]:
date_range = ["{} {}".format(str(date_range_2_)[0:10], str(date_range_1_)[0:10]) for date_range_2_, date_range_1_ in zip(date_range_1, date_range_2)]


#### Full query - DK cross section

In [ ]:
dfTrends_DK_cross_0_50 = trends_fetch_cross(dates=date_range[0:50], terms = term_list(geo='DK'), sleep = 11)

In [ ]:
dfTrends_DK_cross_0_50.to_csv('data/gt/dfTrends_DK_cross_0_50.csv', index=True, sep=';', encoding='utf-8-sig')

In [ ]:
dfTrends_DK_cross_50_75 = trends_fetch_cross(dates=date_range[50:75], terms = term_list(geo='DK'), sleep = 20)

In [ ]:
dfTrends_DK_cross_50_75.to_csv('data/gt/dfTrends_DK_cross_50_75.csv', index=True, sep=';', encoding='utf-8-sig')

In [ ]:
dfTrends_DK_cross_75_100 = trends_fetch_cross(dates=date_range[75:100], terms = term_list(geo='DK'), sleep = 20)

In [ ]:
dfTrends_DK_cross_75_100.to_csv('data/gt/dfTrends_DK_cross_75_100.csv', index=True, sep=';', encoding='utf-8-sig')

In [4]:
dfTrends_DK_cross_100_125 = trends_fetch_cross(dates=date_range[100:125], terms = term_list(geo='DK'), sleep = 20)

Keyterms for each date:   8%|████▍                                                   | 2/25 [14:58<2:52:10, 449.17s/it]

Got blocked at ['"lærernes a-kasse" + "lærernes a kasse"'] for date: 2014-07-01 2014-08-01


Keyterms for each date:  20%|███████████▏                                            | 5/25 [38:30<2:32:53, 458.65s/it]

Got blocked at ['faglig fælles a-kasse + "3f" + "3f a-kasse" + "3f a kasse"'] for date: 2014-10-01 2014-11-01


Keyterms for each date: 100%|███████████████████████████████████████████████████████| 25/25 [3:09:58<00:00, 455.95s/it]


Number of search terms matches number of columns in the dataframe


In [5]:
dfTrends_DK_cross_100_125.to_csv('data/gt/dfTrends_DK_cross_100_125.csv', index=True, sep=';', encoding='utf-8-sig')

In [4]:
dfTrends_DK_cross_125_166 = trends_fetch_cross(dates=date_range[125:], terms = term_list(geo='DK'), sleep = 20)

Keyterms for each date:  15%|████████▏                                               | 6/41 [45:31<4:25:35, 455.31s/it]

Got blocked at ['dagpenge + "dagpenge regler" + dagpengeregler'] for date: 2016-12-01 2017-01-01
Got blocked at ['fyret'] for date: 2016-12-01 2017-01-01


Keyterms for each date:  29%|███████████████▌                                     | 12/41 [1:33:09<3:43:13, 461.84s/it]

Got blocked at ['jobindex + "job index"'] for date: 2017-06-01 2017-07-01


Keyterms for each date:  73%|██████████████████████████████████████▊              | 30/41 [3:50:40<1:23:23, 454.85s/it]

Got blocked at ['jobcenter + jobcentre + "job center" + "job centre"'] for date: 2018-12-01 2019-01-01


Keyterms for each date:  98%|█████████████████████████████████████████████████████▋ | 40/41 [5:07:28<07:34, 454.98s/it]

Got blocked at ['ofir + ofir.dk + "ofir jobportal"'] for date: 2019-10-01 2019-11-01


Keyterms for each date: 100%|███████████████████████████████████████████████████████| 41/41 [5:15:47<00:00, 462.12s/it]


Number of search terms matches number of columns in the dataframe


In [5]:
dfTrends_DK_cross_125_166.to_csv('data/gt/dfTrends_DK_cross_125_166.csv', index=True, sep=';', encoding='utf-8-sig')

#### Merging and exporting cross section

In [9]:
dfTrends_DK_cross_0_50 = pd.read_csv('data/gt/dfTrends_DK_cross_0_50.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])
dfTrends_DK_cross_50_75 = pd.read_csv('data/gt/dfTrends_DK_cross_50_75.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])
dfTrends_DK_cross_75_100 = pd.read_csv('data/gt/dfTrends_DK_cross_75_100.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])
dfTrends_DK_cross_100_125 = pd.read_csv('data/gt/dfTrends_DK_cross_100_125.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])
dfTrends_DK_cross_125_166 = pd.read_csv('data/gt/dfTrends_DK_cross_125_166.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])

In [12]:
dfTrends_DK_cross = pd.concat([dfTrends_DK_cross_0_50, dfTrends_DK_cross_50_75, dfTrends_DK_cross_75_100, dfTrends_DK_cross_100_125, dfTrends_DK_cross_125_166], ignore_index=True)

In [14]:
# Renaming cols
dfTrends_DK_cross.rename(columns= {j : 'GT_DK'+'_'+str(i) +'_cross' for i, j in enumerate(term_list('DK'))}, inplace = True)

In [21]:
#Merging ID
dfTrends_DK_cross = pd.merge(dfTrends_DK_cross, global_id(geo='DK')[['ID', 'trends']], how = 'left', left_on = 'geoCode', right_on = 'trends')

In [22]:
#Exporting
dfTrends_DK_cross.to_csv('DK/data/dfTrends_DK_cross.csv', index=True, sep=';', encoding='utf-8-sig')

## All countries - merging and exporting

In [61]:
dfTrends_DK = pd.read_csv('data/gt/dfTrends_DK_raw.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])
dfTrends_SE = pd.read_csv('data/gt/dfTrends_SE.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])
dfTrends_NO = pd.read_csv('data/gt/dfTrends_NO.csv', sep = ';', encoding = 'utf-8-sig', index_col=0, parse_dates=['date'])

In [62]:
dfTrends_DK['C_ID'] = 'DK'
dfTrends_NO['C_ID'] = 'NO'
dfTrends_SE['C_ID'] = 'SE'

In [63]:
for df in [dfTrends_DK, dfTrends_SE, dfTrends_NO]:
    print(df['C_ID'].unique())
    print(df.shape)
    print(df.isna().sum())
    print('\n')
    

['DK']
(950, 27)
date                                                                                               0
geo                                                                                                0
kontanthjælp + "kontanthjælp sats" + "kontanthjælp satser"                                         0
cv + "cv eksempel" + "cv skabelon"                                                                 0
job + jobopslag + "job opslag"                                                                     0
jobcenter + jobcentre + "job center" + "job centre"                                                0
"ledige job" + "ledige jobs" + "ledig stilling" + "ledige stillinger"                              0
ledig + ledighed                                                                                   0
arbejdsløs + arbejdsløshed                                                                         0
jobindex + "job index"                                                    

In [64]:
#Renaming cols and storing as a dict
for df in [dfTrends_DK, dfTrends_SE, dfTrends_NO]:
    df.rename(columns= {j : 'GT_'+str(df['C_ID'].unique()[0])+'_'+str(i) for i, j in enumerate(term_list(geo=df['C_ID'].unique()[0]))}, inplace = True)
    

#### Dict for GT lookups of terms

In [65]:
# DK = {'GT_'+str(dfTrends_DK['C_ID'].unique()[0])+'_'+str(i) : j for i, j in enumerate(term_list(geo='DK'))}
# SE = {'GT_'+str(dfTrends_SE['C_ID'].unique()[0])+'_'+str(i) : j for i, j in enumerate(term_list(geo='SE'))}
# NO = {'GT_'+str(dfTrends_NO['C_ID'].unique()[0])+'_'+str(i) : j for i, j in enumerate(term_list(geo='NO'))}

In [66]:
# GT_dict_temp = dict(list(DK.items()) + list(SE.items()) + list(NO.items()))

In [67]:
GT_dict = GT_dict()

TypeError: 'dict' object is not callable

Remember to update func for new or removed terms!

Dropping all columns with missings - for now...

In [68]:
GT_dict

{'GT_DK_0': 'kontanthjælp + "kontanthjælp sats" + "kontanthjælp satser"',
 'GT_DK_1': 'cv + "cv eksempel" + "cv skabelon"',
 'GT_DK_2': 'job + jobopslag + "job opslag"',
 'GT_DK_3': 'jobcenter + jobcentre + "job center" + "job centre"',
 'GT_DK_4': '"ledige job" + "ledige jobs" + "ledig stilling" + "ledige stillinger"',
 'GT_DK_5': 'ledig + ledighed',
 'GT_DK_6': 'arbejdsløs + arbejdsløshed',
 'GT_DK_7': 'jobindex + "job index"',
 'GT_DK_8': 'ofir + ofir.dk + "ofir jobportal"',
 'GT_DK_9': 'jobnet + "jobnet.dk" + "jobnet cv"',
 'GT_DK_10': 'akasse + akasser + "a-kasse" + "a-kasser" + "a kasse"',
 'GT_DK_11': 'dagpenge + "dagpenge regler" + dagpengeregler',
 'GT_DK_12': 'dagpengesats + "dagpenge sats" + dagpengesatser + "dagpenge satser"',
 'GT_DK_13': 'akademikernes + "akademikernes a-kasse" + "akademikernes akasse" + "ingeniørernes akasse" + iak',
 'GT_DK_14': 'ase + "ase a-kasse" + "ase akasse"',
 'GT_DK_15': 'bupl + "Børne- og Ungdomspædagogernes Landsforbund" + "Børne- og Ungdomspæ

In [75]:
dfTrends_SE.isna().sum()

date        0
geo         0
GT_SE_0     0
GT_SE_1     0
GT_SE_2     0
GT_SE_5     0
GT_SE_6     0
GT_SE_7     0
GT_SE_8     0
GT_SE_9     0
GT_SE_10    0
GT_SE_11    0
ID          0
trends      0
C_ID        0
dtype: int64

In [71]:
dfTrends_SE = dfTrends_SE[dfTrends_SE['ID']!='Jämtland'].copy()

In [72]:
dfTrends_SE.drop(['GT_SE_4', 'GT_SE_12', 'GT_SE_13', 'GT_SE_3'], axis = 1, inplace = True)

In [74]:
for df in [dfTrends_DK, dfTrends_SE, dfTrends_NO]:
    df.dropna(axis=1, inplace=True)

### Fixing NO - Trøndelag

In [76]:
dfTrends_NO['ID'].unique()

array(['Akershus', 'Aust-Agder', 'Buskerud', 'Finnmark', 'Hedmark',
       'Hordaland', 'Møre og Romsdal', 'Nord-Trøndelag', 'Nordland',
       'Oppland', 'Oslo', 'Østfold', 'Rogaland', 'Sogn og Fjordane',
       'Sør-Trøndelag', 'Telemark', 'Troms', 'Vest-Agder', 'Vestfold'],
      dtype=object)

In [77]:
pop = pd.read_csv('data/other/df_NO_pop_initial.csv', sep = ',')
pop['region'].unique()

array([' Østfold', ' Akershus', ' Oslo', ' Hedmark', ' Oppland',
       ' Buskerud', ' Vestfold', ' Telemark', ' Aust-Agder',
       ' Vest-Agder', ' Rogaland', ' Hordaland', ' Sogn og Fjordane',
       ' Møre og Romsdal', ' Trøndelag', ' Sør-Trøndelag (-2017)',
       ' Nord-Trøndelag (-2017)', ' Nordland', ' Troms - Romsa',
       ' Finnmark - Finnmárku', ' Svalbard'], dtype=object)

In [78]:
pop = pop[(pop['region'] == ' Nord-Trøndelag (-2017)') | (pop['region'] == ' Sør-Trøndelag (-2017)')].copy()
pop = pop[pop['pop'] != 0].sort_values(by = ['region', 'date']).copy()

In [79]:
pop_NORD = pop[(pop['region'] == ' Nord-Trøndelag (-2017)')].set_index('date').drop(['region', 'reg_nr'], axis = 1)

pop_SYD = pop[(pop['region'] == ' Sør-Trøndelag (-2017)')].set_index('date').drop(['region', 'reg_nr'], axis = 1)

pop_NORD['ratio'] = pop_NORD['pop'] / pop_SYD['pop']

ratio = pop_NORD['ratio'].mean()
ratio

0.4537470259083401

In [80]:
pop.groupby('region')['pop'].describe()

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
Nord-Trøndelag (-2017),15.0,131892.933333,3299.855894,127610.0,128881.5,131555.0,134792.5,137233.0
Sør-Trøndelag (-2017),15.0,291151.800000,16363.174056,268188.0,277119.5,290547.0,304476.0,317363.0


In [81]:
test_NORD = dfTrends_NO[(dfTrends_NO['ID'] == 'Nord-Trøndelag')].set_index('date').copy()
test_SYD = dfTrends_NO[(dfTrends_NO['ID'] == 'Sør-Trøndelag')].set_index('date').copy()

In [82]:
dfTrends_NO.columns

Index(['date', 'geo', 'GT_NO_0', 'GT_NO_1', 'GT_NO_2', 'GT_NO_6', 'GT_NO_7',
       'GT_NO_8', 'GT_NO_10', 'GT_NO_11', 'ID', 'trends', 'C_ID'],
      dtype='object')

In [83]:
cols = ['GT_NO_0', 'GT_NO_1', 'GT_NO_2', 'GT_NO_6', 'GT_NO_7',
       'GT_NO_8', 'GT_NO_10', 'GT_NO_11']

In [84]:
for col in cols:
    test_NORD[col] = test_NORD[col]*ratio + (1-ratio) * test_SYD[col]

In [85]:
test_NORD.reset_index()
test_NORD['ID'] = 'Trøndelag'
test_NORD['trends'] = 'NO-50'
test_NORD['geo'] = 'NO-50'

In [86]:
test_NORD.reset_index(inplace=True)

In [87]:
dfTrends_NO = dfTrends_NO[(dfTrends_NO['ID'] != 'Nord-Trøndelag')].copy()
dfTrends_NO = dfTrends_NO[(dfTrends_NO['ID'] != 'Sør-Trøndelag')].copy()

In [88]:
dfTrends_NO = pd.concat([dfTrends_NO, test_NORD], ignore_index=True, sort=False).sort_values(['ID', 'date']).copy()

In [89]:
dfTrends_NO.shape

(3420, 13)

### Concatting

In [90]:
dfTrends = pd.concat([dfTrends_DK, dfTrends_NO, dfTrends_SE], ignore_index=True, sort = False).fillna(value = 0)

In [91]:
dfTrends.sort_values(['geo', 'date'], inplace=True)

In [92]:
dfTrends.shape

(8170, 45)

In [93]:
dfTrends.columns

Index(['date', 'geo', 'GT_DK_0', 'GT_DK_1', 'GT_DK_2', 'GT_DK_3', 'GT_DK_4',
       'GT_DK_5', 'GT_DK_6', 'GT_DK_7', 'GT_DK_8', 'GT_DK_9', 'GT_DK_10',
       'GT_DK_11', 'GT_DK_12', 'GT_DK_13', 'GT_DK_14', 'GT_DK_15', 'GT_DK_16',
       'GT_DK_17', 'GT_DK_18', 'GT_DK_19', 'GT_DK_20', 'GT_DK_21', 'ID',
       'trends', 'C_ID', 'GT_NO_0', 'GT_NO_1', 'GT_NO_2', 'GT_NO_6', 'GT_NO_7',
       'GT_NO_8', 'GT_NO_10', 'GT_NO_11', 'GT_SE_0', 'GT_SE_1', 'GT_SE_2',
       'GT_SE_5', 'GT_SE_6', 'GT_SE_7', 'GT_SE_8', 'GT_SE_9', 'GT_SE_10',
       'GT_SE_11'],
      dtype='object')

### Aggregating to quarterly

In [94]:
temp = dfTrends.set_index('date').index.to_period('Q')

In [95]:
dfTrends['date'] = temp

In [96]:
dfTrends = dfTrends.groupby(['C_ID', 'geo', 'trends', 'ID', 'date'], as_index=False).mean()

In [97]:
dfTrends.shape

(2752, 45)

In [98]:
dfTrends

,C_ID,geo,trends,ID,date,GT_DK_0,GT_DK_1,GT_DK_2,GT_DK_3,GT_DK_4,...,GT_SE_0,GT_SE_1,GT_SE_2,GT_SE_5,GT_SE_6,GT_SE_7,GT_SE_8,GT_SE_9,GT_SE_10,GT_SE_11
0,DK,DK-81,DK-81,North Denmark,2004Q1,0.000000,74.333333,81.000000,0.0,67.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,DK,DK-81,DK-81,North Denmark,2004Q2,36.666667,73.000000,78.666667,0.0,36.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,DK,DK-81,DK-81,North Denmark,2004Q3,59.000000,51.333333,79.000000,9.0,43.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,DK,DK-81,DK-81,North Denmark,2004Q4,0.000000,57.000000,63.333333,18.0,52.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,DK,DK-81,DK-81,North Denmark,2005Q1,42.000000,54.333333,92.666667,14.0,38.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,SE,SE-Y,SE-Y,Västernorrland,2018Q4,0.000000,0.000000,0.000000,0.0,0.000000,...,61.666667,56.333333,74.666667,25.333333,23.666667,32.000000,58.333333,28.333333,24.333333,19.333333
2748,SE,SE-Y,SE-Y,Västernorrland,2019Q1,0.000000,0.000000,0.000000,0.0,0.000000,...,73.666667,67.333333,85.666667,17.333333,22.666667,36.666667,79.000000,28.333333,27.666667,20.333333
2749,SE,SE-Y,SE-Y,Västernorrland,2019Q2,0.000000,0.000000,0.000000,0.0,0.000000,...,61.000000,60.666667,65.666667,30.666667,22.333333,29.666667,62.000000,32.333333,20.333333,18.000000
2750,SE,SE-Y,SE-Y,Västernorrland,2019Q3,0.000000,0.000000,0.000000,0.0,0.000000,...,66.000000,61.666667,73.000000,22.000000,21.333333,40.000000,61.333333,33.666667,17.333333,18.000000


### Exporting

In [99]:
dfTrends_DK.to_csv('data/gt/dfTrends_DK.csv', index=True, sep=';', encoding='utf-8-sig')
dfTrends_NO.to_csv('data/gt/dfTrends_NO.csv', index=True, sep=';', encoding='utf-8-sig')
dfTrends_SE.to_csv('data/gt/dfTrends_SE.csv', index=True, sep=';', encoding='utf-8-sig')

In [ ]:
#For only DK data
# Dropping country col
dfTrends_DK.drop(['C_ID'], axis = 1, inplace=True)
dfTrends_DK.to_csv('DK/data/dfTrends_DK.csv', index=True, sep=';', encoding='utf-8-sig')

In [103]:
# dfTrends.to_csv('data/gt/dfTrends.csv', index=True, sep=';', encoding='utf-8-sig')
dfTrends.to_csv('robustness/data/gt/dfTrends.csv', index=True, sep=';', encoding='utf-8-sig')